<h2><center> Jane Street Market Prediction  - reruns dashboard </center></h2>

<h2><center> <img src="https://upload.wikimedia.org/wikipedia/commons/7/7c/Stock_Bull.jpg" width="400" alt="Stock Market Bull img"></center></h2>

[The leaderboard](https://www.kaggle.com/c/jane-street-market-prediction/leaderboard) for Jane Street Market Prediction competition shows only the final (recent) score for each team. It is interesting to look at public and intermediate reruns' scores.

*NB: team members in dashboard are clickable*

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import json

leaderboard_old = pd.read_csv("../input/jane-street-market-prediction-leaderboard/leaderboard.csv")
leaderboard_old = leaderboard_old.drop(columns=["teamName", "medal", "teamName"]).\
    rename(columns={"final_score": "12th_rerun_score", "final_rank": "12th_rerun_rank"})

teams = pd.read_csv('../input/meta-kaggle/Teams.csv')
submissions = pd.read_csv('../input/meta-kaggle/Submissions.csv')
submissions = submissions[["Id", "PrivateScoreLeaderboardDisplay"]].\
    rename(columns={"PrivateScoreLeaderboardDisplay": "final_score", "Id": "SubmissionId"})
leaderboard_new = teams.loc[teams["CompetitionId"] == 23304, ["Id", "PrivateLeaderboardSubmissionId", "TeamName"]]
leaderboard_new = leaderboard_new[pd.notna(leaderboard_new["PrivateLeaderboardSubmissionId"])].\
    rename(columns={"Id": "teamId", "PrivateLeaderboardSubmissionId": "SubmissionId", "TeamName": "teamName"}).\
    reset_index(drop=True)
leaderboard_new["SubmissionId"] = leaderboard_new["SubmissionId"].astype(np.int64)
leaderboard_new = pd.merge(leaderboard_new, submissions, on="SubmissionId")
del teams, submissions
leaderboard_new["final_rank"] = leaderboard_new["final_score"].rank(ascending=False, method="first").astype(np.int64)

gold_min_score = leaderboard_new[leaderboard_new["final_rank"] == 18]["final_score"].values[0]
silver_min_score = leaderboard_new[leaderboard_new["final_rank"] == 212]["final_score"].values[0]
bronze_min_score = leaderboard_new[leaderboard_new["final_rank"] == 424]["final_score"].values[0]
leaderboard_new["medal"]= leaderboard_new.apply(lambda x: "gold" if x["final_score"] >= gold_min_score
                                         else "silver" if x["final_score"] >= silver_min_score and x["final_score"] < gold_min_score
                                         else "bronze" if x["final_score"] >= bronze_min_score and x["final_score"] < silver_min_score
                                         else np.NaN, 1)
leaderboard = pd.merge(leaderboard_old, leaderboard_new, on="teamId")
leaderboard.to_csv("leaderboard.csv", index=False)
del leaderboard_old, leaderboard_new, gold_min_score, silver_min_score, bronze_min_score

leaderboard["teamMembers"] = leaderboard["teamMembers"].\
    apply(lambda x: x.strip('][').replace('{', '').replace('},', '}').split('}'))
leaderboard["teamMembers"] = leaderboard["teamMembers"].\
    apply(lambda x : [json.loads('{' + el.replace("\'", "\"").replace('\\', "").\
                      replace("r\"s", "r\'s").replace('}', '' + '}') + '}') for el in x if el!=''])
leaderboard["medal"]= leaderboard.apply(lambda x: "https://www.kaggle.com/static/images/medals/competitions/goldl@1x.png" if x["medal"] == "gold"
                                         else "https://www.kaggle.com/static/images/medals/competitions/silverl@1x.png" if x["medal"] == "silver"
                                         else "https://www.kaggle.com/static/images/medals/competitions/bronzel@1x.png" if x["medal"] == "bronze"
                                         else np.NaN, 1)

In [ ]:
leaderboard.loc[:, ["teamName", 
                    "teamMembers", 
                    "public_score", 
                    "1st_rerun_score", 
                    "2nd_rerun_score", 
                    "3rd_rerun_score", 
                    "4th_rerun_score", 
                    "5th_rerun_score", 
                    "6th_rerun_score",
                    "7th_rerun_score",
                    "8th_rerun_score",
                    "9th_rerun_score",
                    "10th_rerun_score",
                    "11th_rerun_score",
                    "12th_rerun_score",
                    "final_score",
                    "public_rank", 
                    "1st_rerun_rank", 
                    "2nd_rerun_rank", 
                    "3rd_rerun_rank", 
                    "4th_rerun_rank", 
                    "5th_rerun_rank", 
                    "6th_rerun_rank",
                    "7th_rerun_rank",
                    "8th_rerun_rank",
                    "9th_rerun_rank",
                    "10th_rerun_rank",
                    "11th_rerun_rank",
                    "12th_rerun_rank",
                    "final_rank",
                    "medal"]].\
    sort_values("final_rank", ascending=True).\
    style.format({
        "teamMembers": lambda x: "".join(f'<div style="float: right; margin: -4px 2px;"><a href="https://www.kaggle.com{xi["profileUrl"]}"><img src="{xi["thumbnailUrl"]}" width="24" height="24" alt="{xi["displayName"]}"></a></div>' for xi in x)
       ,"public_score": '{:,.3f}'.format
       ,"1st_rerun_score": '{:,.3f}'.format
       ,"2nd_rerun_score": '{:,.3f}'.format
       ,"3rd_rerun_score": '{:,.3f}'.format
       ,"4th_rerun_score": '{:,.3f}'.format
       ,"5th_rerun_score": '{:,.3f}'.format
       ,"6th_rerun_score": '{:,.3f}'.format
       ,"7th_rerun_score": '{:,.3f}'.format
       ,"8th_rerun_score": '{:,.3f}'.format
       ,"9th_rerun_score": '{:,.3f}'.format
       ,"10th_rerun_score": '{:,.3f}'.format
       ,"11th_rerun_score": '{:,.3f}'.format
       ,"12th_rerun_score": '{:,.3f}'.format
       ,"final_score": '{:,.3f}'.format
       ,"medal": lambda x: f'<div style="float: right; margin: -4px 2px;"><a href="{x}"><img src="{x}" width="24" height="24" alt="{x}"></a></div>' if x!='' else ''
    }).\
background_gradient(subset=["public_score", 
                            "1st_rerun_score", 
                            "2nd_rerun_score", 
                            "3rd_rerun_score", 
                            "4th_rerun_score", 
                            "5th_rerun_score", 
                            "6th_rerun_score",
                            "7th_rerun_score",
                            "8th_rerun_score",
                            "9th_rerun_score",
                            "10th_rerun_score",
                            "11th_rerun_score",
                            "12th_rerun_score",
                            "final_score"], cmap="Wistia").\
background_gradient(subset=["public_rank", 
                            "1st_rerun_rank", 
                            "2nd_rerun_rank", 
                            "3rd_rerun_rank", 
                            "4th_rerun_rank", 
                            "5th_rerun_rank", 
                            "6th_rerun_rank",
                            "7th_rerun_rank",
                            "8th_rerun_rank",
                            "9th_rerun_rank",
                            "10th_rerun_rank",
                            "11th_rerun_rank",
                            "12th_rerun_rank",
                            "final_rank"], cmap="OrRd").\
hide_index()

### Are there any difference? Reruns score distribution.

In [ ]:
scores = leaderboard.loc[:, ["public_score", 
                             "1st_rerun_score", 
                             "2nd_rerun_score", 
                             "3rd_rerun_score", 
                             "4th_rerun_score", 
                             "5th_rerun_score", 
                             "6th_rerun_score",
                             "7th_rerun_score",
                             "8th_rerun_score",
                             "9th_rerun_score",
                             "10th_rerun_score",
                             "11th_rerun_score",
                             "12th_rerun_score",
                             "final_score"]]
sns.displot(scores.unstack().reset_index(name='score').rename(columns={"level_0": "rerun"}).drop(columns=["level_1"]), 
            x="score", hue="rerun", kind="kde", height=15)
plt.show()

In [ ]:
scores.describe().style.format('{:,.2f}'.format)